## YouTube Video → AI Study Notes

In [93]:
import re
import torch
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer

In [94]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [95]:
def extract_video_id(url):
    """Extracts video ID from different YouTube URL formats."""
    # We use Regex to hunt for the 11-character ID after 'v=' or 'youtu.be/'
    match = re.search(r"(?:v=|youtu\.be/)([a-zA-Z0-9_-]{11})", url)
    return match.group(1) if match else None

def get_transcript(video_id):
    """Fetch transcript using the NEW API format."""
    try:
        api = YouTubeTranscriptApi()
        # The .fetch method grabs the subtitle object list
        transcript = api.fetch(video_id)
        # We join the list into a single long string of text
        return " ".join([t.text for t in transcript])

    except TranscriptsDisabled:
        return "Error: Transcripts are disabled for this video."
    except NoTranscriptFound:
        return "Error: No transcript found for this video."
    except Exception as e:
        return f"Error: {str(e)}"

In [96]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Check if GPU (CUDA) available 
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "google/flan-t5-base"


tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

Loading weights: 100%|██████████| 282/282 [00:00<00:00, 611.42it/s, Materializing param=shared.weight]                                                       
The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


In [97]:
def summarize_chunk(text_chunk):
    """
    Summarizes a text chunk with enhanced settings for longer, more informative summaries.
    """
    
    prompt = f"""
You are an expert content summarizer. Summarize the following text clearly, accurately, and in detail. 
Your summary should be easy to understand, concise, and preserve all important points. 
Organize the summary logically and use complete sentences. Highlight key topics, main arguments, and any critical examples.

{text_chunk}"""

    # Tokenize 
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1024
    ).to(device)

    # Generate summary
    summary_ids = model.generate(
        **inputs,
        max_new_tokens=300,        
        num_beams=6,               
        length_penalty=1.2,        
        min_length=100,            
        no_repeat_ngram_size=3,    
        early_stopping=True,
        temperature=0.7,           
        top_p=0.9                  
    )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [98]:
def chunk_text_with_overlap(text, chunk_size=1200, overlap=200):
    """
    Splits text into chunks with overlap.
    - chunk_size: approx number of tokens per chunk
    - overlap: approx number of tokens to overlap between chunks
    """
    sentences = text.split(". ")
    chunks = []
    current_chunk = []

    token_count = 0

    for sentence in sentences:
        sentence_tokens = sentence.split()  
        if token_count + len(sentence_tokens) <= chunk_size:
            current_chunk.extend(sentence_tokens)
            token_count += len(sentence_tokens)
        else:
            
            chunks.append(" ".join(current_chunk))
            
            # Start new chunk with overlap
            current_chunk = current_chunk[-overlap:] if overlap < len(current_chunk) else current_chunk
            current_chunk.extend(sentence_tokens)
            token_count = len(current_chunk)

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks


In [99]:
def generate_video_notes(video_url):
    print(f"\nProcessing video: {video_url}")

    video_id = extract_video_id(video_url)
    if not video_id:
        print("Invalid YouTube URL.")
        return

    print("Fetching transcript...")
    transcript = get_transcript(video_id)

    if transcript.startswith("Error"):
        print(transcript)
        return

    print("🔪 Chunking transcript...")
    chunks = chunk_text(transcript)
    print(f"   -> {len(chunks)} chunks created.")

    print("Generating AI notes...")
    notes = []

    # Loop through chunks and summarize each one
    for i, chunk in enumerate(chunks):
        print(f"   Summarizing chunk {i+1}/{len(chunks)}...")
        summary = summarize_chunk(chunk)
        notes.append(f"- {summary}")

    print("\n" + "="*50)
    print("AI GENERATED NOTES")
    print("="*50)
    print("\n".join(notes))


if __name__ == "__main__":
    url = input("Paste YouTube URL: ")
    generate_video_notes(url)


Processing video: https://youtu.be/lUljxdkolK8?si=4IfjuZiDMQC6F-nW
Fetching transcript...
🔪 Chunking transcript...
   -> 9 chunks created.
Generating AI notes...
   Summarizing chunk 1/9...
   Summarizing chunk 2/9...
   Summarizing chunk 3/9...
   Summarizing chunk 4/9...
   Summarizing chunk 5/9...
   Summarizing chunk 6/9...
   Summarizing chunk 7/9...
   Summarizing chunk 8/9...
   Summarizing chunk 9/9...

AI GENERATED NOTES
- Physicists are able to reversibly change the dynamics of atoms, molecules, and spheres. Here's a look at how physicians can do just the same thing — and generate new data in the process — using physics as a way of resolving chaos in the world of generative AI. Phylogenetic phylogeny is based on quantum mechanics, and it's one of the most important fields in the field of theoretical cosmodynamics. It's also one of physics' most important applications.
- I'm going to make a two-part video focusing on diffusion models. So let's take a closer look and see what 